In [12]:
import numpy as np
import qutip

In [19]:
data = np.load('results/data/back_action/' \
'x/Qinp/QRC/qubit/L4_Js1_h3_W0.01_dt0.5_V1_D1_Nrep1_MeasStr_0.2' \
'/Iter_0.npz', allow_pickle=True)

In [20]:
data['obs'].shape

(4000, 10)

In [24]:
a = b = c =None

In [26]:
print(a)

None
